# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [24]:
# Do all imports and installs here
import pandas as pd
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, dayofweek, monotonically_increasing_id
from pyspark.sql.types import TimestampType

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [17]:
# Read in the data here
# fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
# df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

imm_data = 'immigration_data_sample.csv'
df = pd.read_csv(imm_data)
df.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [18]:
df_imm = df[["airline"]].drop_duplicates().sort_values('airline')
pd.set_option('display.max_rows', 10)
df_imm

,airline
1,*GA
36,2D
368,3U
152,4M
220,4O
...,...
901,WS
29,Y4
652,YX
973,ZX


In [19]:
df_cic = df.duplicated(subset=['cicid'])
df_cic

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool

In [20]:
# df_dup = df[["cicid",""]]
df[df.duplicated(subset=['cicid'])]

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype


In [28]:
city_demo = 'us-cities-demographics.csv'
headers = ['city', 'state', 'median_age', 'male_population', 'female_population', \
                 'total_population', 'no_of_veterans', 'no_of_foreign_born', \
                 'avg_househld_size', 'state_code', 'race', 'count']
df_city = pd.read_csv(city_demo, sep=';', names=headers, skiprows=1)
df_city.head()

,city,state,median_age,male_population,female_population,total_population,no_of_veterans,no_of_foreign_born,avg_househld_size,state_code,race,count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [29]:
df_city['race'].drop_duplicates()

0                   Hispanic or Latino
1                                White
2                                Asian
3            Black or African-American
5    American Indian and Alaska Native
Name: race, dtype: object

In [32]:
ds = df_city.sort_values('city')
ds.head(10)

,city,state,median_age,male_population,female_population,total_population,no_of_veterans,no_of_foreign_born,avg_househld_size,state_code,race,count
2727,Abilene,Texas,31.3,65212.0,60664.0,125876,9367.0,8129.0,2.64,TX,Asian,2929
1403,Abilene,Texas,31.3,65212.0,60664.0,125876,9367.0,8129.0,2.64,TX,Hispanic or Latino,33222
1533,Abilene,Texas,31.3,65212.0,60664.0,125876,9367.0,8129.0,2.64,TX,White,95487
245,Abilene,Texas,31.3,65212.0,60664.0,125876,9367.0,8129.0,2.64,TX,American Indian and Alaska Native,1813
2880,Abilene,Texas,31.3,65212.0,60664.0,125876,9367.0,8129.0,2.64,TX,Black or African-American,14449
2175,Akron,Ohio,38.1,96886.0,100667.0,197553,12878.0,10024.0,2.24,OH,Asian,9033
560,Akron,Ohio,38.1,96886.0,100667.0,197553,12878.0,10024.0,2.24,OH,Black or African-American,66551
1100,Akron,Ohio,38.1,96886.0,100667.0,197553,12878.0,10024.0,2.24,OH,White,129192
2325,Akron,Ohio,38.1,96886.0,100667.0,197553,12878.0,10024.0,2.24,OH,Hispanic or Latino,3684
1972,Akron,Ohio,38.1,96886.0,100667.0,197553,12878.0,10024.0,2.24,OH,American Indian and Alaska Native,1845


In [21]:
airport_codes = 'airport-codes_csv.csv'
df_codes = pd.read_csv(airport_codes)
df_codes.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [22]:
df_airport=df_codes['ident'].drop_duplicates()
pd.set_option('display.max_rows', 30)
df_airport

0            00A
1           00AA
2           00AK
3           00AL
4           00AR
5           00AS
6           00AZ
7           00CA
8           00CL
9           00CN
10          00CO
11          00FA
12          00FD
13          00FL
14          00GA
          ...   
55060       ZYLS
55061       ZYMD
55062       ZYMH
55063       ZYQQ
55064       ZYSQ
55065       ZYTH
55066       ZYTL
55067       ZYTN
55068       ZYTX
55069       ZYYJ
55070       ZYYK
55071       ZYYY
55072    ZZ-0001
55073    ZZ-0002
55074       ZZZZ
Name: ident, Length: 55075, dtype: object

In [23]:
df_iata=df_codes['iata_code'].drop_duplicates()
# pd.set_option('display.max_rows', 30)
df_iata

0        NaN
223      UTK
440      OCA
594      PQS
673      CSE
1088     JCY
1402     PMX
1438     WLR
1555     NUP
1574     PTC
1722     ICY
1758     PPV
1984     KKK
2170     MHS
2383     GCT
        ... 
55054    JIL
55055    JMU
55056    JSJ
55057    JXA
55058    JNZ
55059    LDS
55060    YUS
55061    MDG
55063    NDG
55064    YSQ
55066    DLC
55067    TNH
55068    SHE
55069    YNJ
55070    YKH
Name: iata_code, Length: 9043, dtype: object

In [2]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


Exception: Java gateway process exited before sending its port number

In [11]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.